In [1]:
import glob
import os
import shutil

from astropy import stats
from astropy.io import fits
import numpy as np
import pandas as pd

In [2]:
postflash_data = pd.read_pickle('/Volumes/wfc3u/final_postflash_2021/notebooks/Fwfc3u/final_postflash_2021/notebooks/Feb_2022_all_postflash_flc.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_all_postflash_flc.pkl'

In [3]:
postflash_data[0:10]

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time
rootname,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5
idus35y6q,idus35y6q_flc.fits,15573,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-08-15,10:09:26,A,7310,100.0,MED,False,0.5
ie1b44diq,ie1b44diq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-12-09,09:02:57,B,22,0.3,MED,False,100.0
idp531a7q,idp531a7q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5
idp543c4q,idp543c4q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2017-12-11,05:29:33,B,12,4.7,LOW,False,0.5


In [4]:
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
sub_paths = subarray_pf.path.tolist()
all_paths = postflash_data.path.tolist()
print (all_paths)

['/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/icha24imq/icha24imq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp530aiq/idp530aiq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ieg845feq/ieg845feq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/icrz19f7q/icrz19f7q_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idus35y6q/idus35y6q_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ie1b44diq/ie1b44diq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp531a7q/idp531a7q_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp543c4q/idp543c4q_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idus10qfq/idus10qfq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ielu12ppq/ielu12ppq_flc.fits', '/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp503bnq/

In [5]:
means = []
sig_means = []
medians = []
sig_medians = []
stds = []
sig_stds = []

for f in all_paths:
    path = f
    print(path)
    data = fits.getdata(path)
    data_clip = stats.sigma_clip(data, 3)
    
    mean = np.mean(data)
    sig_mean = np.mean(data_clip)
    median = np.median(data)
    sig_median = np.median(data_clip)
    std = np.std(data)
    sig_std = np.std(data_clip)
    
    
    means.append(mean)
    sig_means.append(sig_mean)
    medians.append(median)
    sig_medians.append(sig_median)
    stds.append(std)
    sig_stds.append(sig_std)


/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits


/Users/cmartlin/miniconda3/envs/ql_testing/lib/python3.6/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/icha24imq/icha24imq_flc.fits


/Users/cmartlin/miniconda3/envs/ql_testing/lib/python3.6/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp530aiq/idp530aiq_flc.fits


...

/Users/cmartlin/miniconda3/envs/ql_testing/lib/python3.6/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/id0o07hvq/id0o07hvq_flc.fits


/Users/cmartlin/miniconda3/envs/ql_testing/lib/python3.6/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idp529zwq/idp529zwq_flc.fits


/Users/cmartlin/miniconda3/envs/ql_testing/lib/python3.6/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [7]:
dict = {'path': all_paths, 'mean': means, 'sig_mean': sig_means,'median': medians, 'sig_median': sig_medians, 'std': stds, 'sig_std': sig_stds}
path_and_stats_sub = pd.DataFrame(dict)

all_stats = pd.merge(left=postflash_data, right=path_and_stats_sub, left_on='path', right_on='path').set_index(postflash_data.index)

In [8]:
all_stats['norm_by_dur'] = (all_stats['sig_mean']/all_stats['flash_dur'])
all_stats.to_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')
all_stats = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')
print (all_stats)

                     filename  propid  \
rootname                                
ic6de2u9q  ic6de2u9q_flc.fits   13078   
icha24imq  icha24imq_flc.fits   13560   
idp530aiq  idp530aiq_flc.fits   14984   
ieg845feq  ieg845feq_flc.fits   16398   
icrz19f7q  icrz19f7q_flc.fits   14006   
...                       ...     ...   
icrz24ubq  icrz24ubq_flc.fits   14006   
ieg819ldq  ieg819ldq_flc.fits   16398   
ic6de3riq  ic6de3riq_flc.fits   13078   
id0o07hvq  id0o07hvq_flc.fits   14372   
idp529zwq  idp529zwq_flc.fits   14984   

                                                        path     obsdate  \
rootname                                                                   
ic6de2u9q  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2013-03-09   
icha24imq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2014-06-29   
idp530aiq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2018-06-22   
ieg845feq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2020-12-14   
icrz19f7q 

In [9]:
 all_stats['datetime'] = pd.to_datetime(all_stats['obsdate'] + ' ' + all_stats['obstime'])
 all_stats.to_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')